# Web Scrapping lab

In this lab you will scrappe this [website](https://books.toscrape.com/) of books.

You have to create a Pandas DataFrame with all the books listed in the page. Each row of the DataFrame should contain information of each book. In particular, the DataFrmae must contain:

* category
* title
* price
* stock availability
* star rating (number of stars)
* description
* UPC

Happy scrapping!



# Server verification

Load the needed libraries, and make sure thar you can obtain the correct status code.

In [1]:
# Your code here
from bs4 import BeautifulSoup
import requests

# Book categories

Create the code to collect the **relative urls** from the left panel to obtain a list with all the book categories.

In [2]:
url_lab = "https://books.toscrape.com/"
response = requests.get(url)
html_obtenido = response.text


NameError: name 'url' is not defined

In [ ]:
soup = BeautifulSoup(html_obtenido, 'html.parser')

In [ ]:
# Create the code to collect the relative urls from the left panel to obtain a list with all the book categories.

cat_section = soup.find('ul', class_='nav-list')
cat_links = categories_section.find_all('a')

In [ ]:
category_urls = []
for link in cat_links:
    href = link.get('href')
    if href:
       category_urls.append(href.strip())

In [ ]:
for url in category_urls:
    print(url)

In [ ]:
# Your code here

# Books in a given category

Use. web scrapping and list comprehension to obtain the **absolute** url of each book to be scraped

In [ ]:
# Your code here
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

base_url = "http://books.toscrape.com/"

# Book details

Create a Python function that given a book_url as an input returns a dictionary with the following structure:

```Python
{"Title": title, "Price": price, "Availability": availability, "Rating": rating, "Description": description, "UPC": upc}
```

where `description` should contain the book's summary given in the Product description, and the values are the book's associated information.

In [ ]:
# Your code here

def extract_book_details(book_url):
    response = requests.get(book_url)  # Send a GET request to the book URL
    soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
# extrct the book "title"
title = soup.find('h1').get_text()
title

In [ ]:
# the book price
price = soup.find('p', class_='price_color').get_text()
price

In [ ]:
# availability
availability = soup.find('p', class_='instock availability').get_text(strip=True)
availability

In [ ]:
# rating information
rating_inf = soup.find('p', class_='star-rating')
rating = rating_inf['class'][1] if rating_inf else "No rating"
rating

In [ ]:
# description
description_element = soup.find('meta', {'name': 'description'})
description = description_element['content'].strip() if description_element else "No description available."
description

In [ ]:
# UPC universal product code
upc_cat = soup.find('th', string='UPC')
upc = upc_cat.find_next_sibling('td').get_text()if upc_cat else "No UPC available"
upc


# Collect and store all the information from the books in a Pandas DataFrame

Start with the following dictionary:

```python
books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": [] }
```

Then, iterate over all the categories and all the books in a given category to collect any book information using the previous function. Fill the previous dictionary with the information about each book.

Show the first five rows of the previous final Pandas DataFrame.

Tip: You can use the function `tqdm` from the library `tqdm` to show a progress bar if in iterable of a for loop as shown below :wink: :

```python
from tqdm import tqdm

for elem in tqdm(iterable):
    # some code
```





In [ ]:
from tqdm import tqdm

# Your code here

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from tqdm import tqdm

In [ ]:
base_url = "http://books.toscrape.com/"

In [ ]:
def get_category_urls():
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    categories = soup.find('ul', class_='nav-list').find('ul').find_all('a')
    category_urls = {category.get_text(strip=True): urljoin(base_url, category['href']) for category in categories}
    return category_urls

In [ ]:
def get_book_urls(category_url):
    book_urls = []
    while category_url:
        response = requests.get(category_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        books = soup.find_all('h3')
        book_urls += [urljoin(base_url, book.find('a')['href']) for book in books]

In [ ]:
 # Find the next page URL if it exists
next_page = soup.find('li', class_='next')
category_url = urljoin(base_url, next_page.find('a')['href']) if next_page else None

In [ ]:
# Function to extract book details
def extract_book_details(book_url):
    response = requests.get(book_url)
    soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
# Extract details
title = soup.find('h1').get_text()
price = soup.find('p', class_='price_color').get_text()
availability = soup.find('p', class_='instock availability').get_text(strip=True)
rating_element = soup.find('p', class_='star-rating')
rating = rating_element['class'][1] if rating_element else "No rating"
description_element = soup.find('meta', {'name': 'description'})
description = description_element['content'].strip() if description_element else "No description available."
upc_element = soup.find('th', string='UPC')
upc = upc_element.find_next_sibling('td').get_text() if upc_element else "No UPC available."


In [ ]:
# Loop through each category and each book within that category
category_urls = get_category_urls()

for category_name, category_url in tqdm(category_urls.items(), desc="Processing categories"):
    book_urls = get_book_urls(category_url)
    for book_url in tqdm(book_urls, desc=f"Processing books in {category_name}", leave=False):
        book_details = extract_book_details(book_url)
        # Append book details to the dictionary
        for key, value in book_details.items():
            books_dict[key].append(value)
        books_dict["Category"].append(category_name)
books_df = pd.DataFrame(books_dict)
print(books_df.head())